# Cleaning and Examining Data

In [36]:
import pandas
import seaborn as sns
import matplotlib.pyplot as plt

df = pandas.DataFrame.from_csv('train.tsv', sep='\t')

In [37]:
# looks for any commas, periods, and apostrophes with any leading or following spaces
remove_punct = r'(( )*((\.)|(\'s*)|(,))( )*)'

# remove leading and trailing spaces
remove_spaces = r'(^( )*)|(( )*$)'

# replaces the pattern mentioned above with a single space
df['Phrase'] = df['Phrase'].str.replace(remove_punct, ' ')
df = df[df['Phrase'] != ' ']
df['Phrase'] = df['Phrase'].str.replace(remove_spaces, '')

In [38]:
B = []
before = ""
for r in df['Phrase']:
    if r != before:
        B.append(True)
        before = r
    else:
        B.append(False)
df = df[B]

In [39]:
i = 0
for col in df['Phrase'].head(80):
    print i, col
    print
    i += 1

0 A series of escapades demonstrating the adage that what is good for the goose is also good for the gander some of which occasionally amuses but none of which amounts to much of a story

1 A series of escapades demonstrating the adage that what is good for the goose

2 A series

3 A

4 series

5 of escapades demonstrating the adage that what is good for the goose

6 of

7 escapades demonstrating the adage that what is good for the goose

8 escapades

9 demonstrating the adage that what is good for the goose

10 demonstrating the adage

11 demonstrating

12 the adage

13 the

14 adage

15 that what is good for the goose

16 that

17 what is good for the goose

18 what

19 is good for the goose

20 is

21 good for the goose

22 good

23 for the goose

24 for

25 the goose

26 goose

27 is also good for the gander some of which occasionally amuses but none of which amounts to much of a story

28 is also

29 also

30 good for the gander some of which occasionally amuses but none of which 

In [52]:
class Node:
    def __init__(self, phrase, parent, left, right):
        self.parent = parent
        self.left = left
        self.right = right
        self.phrase = phrase
        self.node = 0

In [60]:
phr = []
phr2 = []
top80 = df['Phrase'].head(80)
top80_senti = df['Sentiment'].head(80)

j = 0
for r1, r2 in zip(top80, top80_senti):
    if j > 59 and j <= 74:
        phr.append(r1)
        phr2.append((r1, r2))
    j += 1
print phr2

[('This quiet introspective and entertaining independent', 3), ('This', 2), ('quiet introspective and entertaining independent', 4), ('quiet introspective and entertaining', 3), ('quiet', 2), ('introspective and entertaining', 3), ('introspective and', 3), ('introspective', 2), ('and', 2), ('entertaining', 4), ('independent', 2), ('is worth seeking', 3), ('is worth', 2), ('worth', 2), ('seeking', 2)]


In [71]:
import pydot

colors = {1: "#F183B3", 2: "#E26E6E", 3: "#DB1F2A", 4: "#FF1100", 5: "#CA002A"}

def make_phrase_ptree(root, phrases, graph):
    if len(phrases) == 0:
        print "error: ran out of phrases"
        return
    if ' ' not in root.phrase:
        make_phrase_ptree(root.parent, phrases, graph)
    elif root.left == 0:
        root.left = Node(phrases[0], root, 0, 0)
        edge = pydot.Edge(root.phrase, phrases[0])
        graph.add_edge(edge)
        make_phrase_ptree(root.left, phrases[1:], graph)
    elif root.right == 0:
        root.right = Node(phrases[0], root, 0, 0)
        edge = pydot.Edge(root.phrase, phrases[0])
        graph.add_edge(edge)
        make_phrase_ptree(root.right, phrases[1:], graph)
    elif root.parent == 0:
        return root
    else:
        make_phrase_ptree(root.parent, phrases, graph)
        
def make_sentiment_ptree(root, phrases, graph):
    print phrases[0]
    if len(phrases) == 0:
        print "error: ran out of phrases"
        return
    if ' ' not in root.phrase:
        print 'got here'
        make_sentiment_ptree(root.parent, phrases, graph)
    elif root.left == 0:
        root.left = Node(phrases[0][0], root, 0, 0)
        root.left.node = pydot.Node(phrases[0][0], style="filled", fillcolor=colors[phrases[0][1]])
        graph.add_node(root.left.node)
        graph.add_edge(pydot.Edge(root.node, root.left.node))
        make_sentiment_ptree(root.left, phrases[1:], graph)
    elif root.right == 0:
        root.right = Node(phrases[0][0], root, 0, 0)
        root.right.node = pydot.Node(phrases[0][0], style="filled", fillcolor=colors[phrases[0][1]])
        graph.add_node(root.right.node)
        graph.add_edge(pydot.Edge(root.node, root.right.node))
        make_sentiment_ptree(root.left, phrases[1:], graph)
    elif root.parent == 0:
        return root
    else:
        make_sentiment_ptree(root.parent, phrases, graph)

In [72]:
root = Node("This quiet introspective and entertaining independent is worth seeking", 0, 0, 0)
grph = pydot.Dot(graph_type='graph')
grph2 = pydot.Dot(graph_type='digraph')
root.node = pydot.Node(phr2[0][0], style="filled", fillcolor="green")
grph2.add_node(root.node)
#root2 = make_phrase_ptree(root, phr, grph)
root3 = make_sentiment_ptree(root, phr2, grph2)

('This quiet introspective and entertaining independent', 3)
('This', 2)
('quiet introspective and entertaining independent', 4)
got here
('quiet introspective and entertaining independent', 4)
('quiet introspective and entertaining', 3)
got here
('quiet introspective and entertaining', 3)
('quiet introspective and entertaining', 3)
('quiet', 2)
('quiet', 2)


In [66]:
grph2.write_png('ex.png')

True

In [102]:
import nltk
single_words = df[(df['Phrase'].str.contains(' ') == False)]
#a = nltk.word_tokenize("What is your name?")
#print single_words['Phrase']
a = single_words['Phrase'].tolist()
for x in a:
    if x.isdigit():
        print x
#print nltk.pos_tag(a)
#nltk.pos_tag(a)

7
25
60
3000
300
42
2002
1915
10
100
1990
1972
3
20
2
90
86
1975
9
120
1920
1971
1899
1790
30
48
71
2000
2
1957
1
15
78
170
2
110
8
2001
1953
84
95
451
1999
12
80
2002
1984
1973
1933
1958
1950
13
1995
51
5
11
77
163
83
4
1989
270
75
1952
94
2455
40
93
65
127
295
105
1986
87
146
101
1970
1994
18
1938
89
1959
1997
2002
26
72
140
53
6
1937
104
1962
1980
1992
22
1987
52
85
1960
19
98
50
99
1991
66
1993
800
96
88
1998
1979
